In [1]:
DEFAULT_INVERTER = {
    'Vac': 277,  # Nominal AC voltage (V)
    'Pso': 28.358692,  # Power consumption during operation (W)
    'Paco': 2500.0,  # Maximum AC power output (W)
    'Pdco': 2592.4729,  # Maximum DC power output (W)
    'Vdco': 360.0,  # Nominal DC voltage (V)
    'C0': -0.000008,  # Curvature between AC power and DC power (1/W)
    'C1': -0.000045,  # Coefficient of `Pdco` variation with DC input voltage (1/V)
    'C2': 0.00041,  # Coefficient of inverter power consumption loss variation with DC input voltage (1/V)
    'C3': -0.002524,  # Coefficient of C0 variation with DC input voltage (1/V)
    'Pnt': 0.5,  # Inverter night time loss (kW)
    'Vdcmax': 416.0,  # Maximum DC voltage (V)
    'Idcmax': 7.201314,  # Maximum DC current (A)
    'Mppt_low': 100.0,  # Minimum MPPT DC voltage (V)
    'Mppt_high': 416.0,  # Maximum MPPT DC voltage (V)
    'CEC_Date': None,  # CEC date (if applicable, None indicates missing data)
    'CEC_Type': 'Utility Interactive',  # Type of CEC classification
    'Name': 'ABB__UNO_2_5_I_OUTD_S_US__277V_'  # Name of the inverter
}

DEFAULT_MODULE = {
    'Name': 'Sunpreme_Inc__SNPM_GxB_510',  # Name of the solar module
    'Technology': 'Thin Film',               # Type of technology used in the solar module
    'BIPV': 'N',                         # Indicates if the module is bifacial (Y/N)
    'STC': 509.97,                          # Standard Test Conditions power output (W)
    'PTC': 479.6,                           # PTC (PVUSA Test Conditions) power output (W)
    'A_c': 2.591,                           # Area of the solar panel (m²)
    'Length': 1.981,                        # Length of the solar panel (m)
    'Width': 1.308,                         # Width of the solar panel (m)
    'N_s': 96,                              # Number of cells in series
    'I_sc_ref': 9.4,                        # Short circuit current (A) at STC
    'V_oc_ref': 74.7,                       # Open circuit voltage (V) at STC
    'I_mp_ref': 8.9,                        # Max Power Current (A) at STC
    'V_mp_ref': 57.3,                       # Max power voltage (V) at STC
    'alpha_sc': 0.00094,                    # Short circuit current change per degree Celsius (A/°C)
    'beta_oc': -0.19422,                    # Open circuit voltage change per degree Celsius (V/°C)
    'T_NOCT': 45.5,                         # Module NOCT (Nominal Operating Cell Temperature) rating (°C)
    'a_ref': 2.41017,                       # Ideality factor (V) from CEC module database
    'I_L_ref': 9.40894,                     # Reference light current (A)
    'I_o_ref': 0.0,                         # Reference diode saturation current (A)
    'R_s': 1.135045,                        # Reference series resistance (Ω)
    'R_sh_ref': 1193.327026,                # Reference shunt resistance (Ω)
    'Adjust': -20.561962,                   # Temperature coefficient adjustment factor
    'gamma_r': -0.3,                        # Gamma (%/°C) indicating performance degradation with temperature
    'Version': 'SAM 2018.11.11 r2',         # Version of the module or simulation software
    'Date': '1/3/2019'                      # Manufacture date of the module
}


In [2]:
import pvlib

sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
cec_inverters=pvlib.pvsystem.retrieve_sam('CECInverter')

In [3]:
# select solar module with smallest difference
# transpose the df
sandia_mods=sandia_modules.T
sandia_mods['diff_Isco']=DEFAULT_MODULE['I_sc_ref']-sandia_mods['Isco']
sandia_mods['percent_diff_Isco']=sandia_mods['diff_Isco']/DEFAULT_MODULE['I_sc_ref']
sandia_mods['diff_Voc']=DEFAULT_MODULE['V_oc_ref']-sandia_mods['Voco']
sandia_mods['percent_diff_Voc']=sandia_mods['diff_Voc']/DEFAULT_MODULE['V_oc_ref']
sandia_mods['diff_Imp']=DEFAULT_MODULE['I_mp_ref']-sandia_mods['Impo']
sandia_mods['percent_diff_Imp']=sandia_mods['diff_Imp']/DEFAULT_MODULE['I_mp_ref']
sandia_mods['diff_Vmp']=DEFAULT_MODULE['V_mp_ref']-sandia_mods['Vmpo']
sandia_mods['percent_diff_Vmp']=sandia_mods['diff_Vmp']/DEFAULT_MODULE['V_mp_ref']

# we can assign weighting score here if necessary
sandia_mods['total_diff']=(sandia_mods['percent_diff_Imp'].abs()
                           +sandia_mods['percent_diff_Isco'].abs()
                           +sandia_mods['percent_diff_Voc'].abs()
                           +sandia_mods['percent_diff_Vmp'].abs())
indexMin=sandia_mods['total_diff'].idxmin()
indexMin


'SolFocus_SF_1100S_CPV_28__330____2010_'

In [4]:
sandia_mods.loc[indexMin]

Vintage                                                           2010
Area                                                             1.502
Material                                                          GaAs
Cells_in_Series                                                     20
Parallel_Strings                                                     1
Isco                                                           8.05057
Voco                                                           62.1078
Impo                                                           7.69405
Vmpo                                                           53.7042
Aisc                                                           0.00057
Aimp                                                          0.000227
C0                                                              1.0003
C1                                                             -0.0003
Bvoco                                                         -0.09498
Mbvoc 

In [5]:
import pvlib
cec_modules=pvlib.pvsystem.retrieve_sam('CECMod')

In [11]:
cecmods=cec_modules.T
cecmods[cecmods['STC']>500]

,Technology,Bifacial,STC,PTC,A_c,Length,Width,N_s,I_sc_ref,V_oc_ref,...,a_ref,I_L_ref,I_o_ref,R_s,R_sh_ref,Adjust,gamma_r,BIPV,Version,Date
Sunpreme_Inc__SNPM_GxB_510,Thin Film,1,509.97,479.6,2.591,1.981,1.308,96,9.4,74.7,...,2.41017,9.40894,0.0,1.135045,1193.327026,-20.561962,-0.3,N,SAM 2018.11.11 r2,1/3/2019


In [14]:
sandia_modules

,Advent_Solar_AS160___2006_,Advent_Solar_Ventura_210___2008_,Advent_Solar_Ventura_215___2009_,Aleo_S03_160__2007__E__,Aleo_S03_165__2007__E__,Aleo_S16_165__2007__E__,Aleo_S16_170__2007__E__,Aleo_S16_175__2007__E__,Aleo_S16_180__2007__E__,Aleo_S16_185__2007__E__,...,Panasonic_VBHN235SA06B__2013_,Trina_TSM_240PA05__2013_,Hanwha_HSL60P6_PA_4_250T__2013_,Suniva_OPT300_72_4_100__2013_,Canadian_Solar_CS6X_300M__2013_,LG_LG290N1C_G3__2013_,Sharp_NDQ235F4__2013_,Solar_Frontier_SF_160S__2013_,SolarWorld_Sunmodule_250_Poly__2013_,Silevo_Triex_U300_Black__2014_
Vintage,2006,2008,2009,2007 (E),2007 (E),2007 (E),2007 (E),2007 (E),2007 (E),2007 (E),...,2013,2013,2013,2013,2013,2013,2013,2013,2013,2014
Area,1.312,1.646,1.646,1.28,1.28,1.378,1.378,1.378,1.378,1.378,...,1.26,1.63,1.65,1.93,1.91,1.64,1.56,1.22,1.68,1.68
Material,mc-Si,mc-Si,mc-Si,c-Si,c-Si,mc-Si,mc-Si,mc-Si,mc-Si,mc-Si,...,a-Si / mono-Si,mc-Si,mc-Si,c-Si,c-Si,c-Si,mc-Si,CIS,mc-Si,c-Si
Cells_in_Series,72,60,60,72,72,50,50,50,50,50,...,72,60,60,72,72,60,60,172,60,96
Parallel_Strings,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Isco,5.564,8.34,8.49,5.1,5.2,7.9,7.95,8.1,8.15,8.2,...,5.8738,8.8449,8.5935,8.5753,8.6388,9.8525,8.6739,2.0259,8.3768,5.771
Voco,42.832,35.31,35.92,43.5,43.6,30.0,30.1,30.2,30.3,30.5,...,52.0042,36.8926,36.8075,44.2921,43.5918,39.6117,36.8276,112.5048,36.3806,68.5983
Impo,5.028,7.49,7.74,4.55,4.65,7.08,7.23,7.38,7.53,7.67,...,5.5383,8.2955,8.0822,7.963,8.1359,9.2473,8.1243,1.8356,7.6921,5.383
Vmpo,32.41,27.61,27.92,35.6,35.8,23.3,23.5,23.7,23.9,24.1,...,43.1204,29.066,29.2011,35.0837,34.9531,31.2921,29.1988,86.6752,28.348,55.4547
Aisc,0.000537,0.00077,0.00082,0.0003,0.0003,0.0008,0.0008,0.0008,0.0008,0.0008,...,0.0005,0.0004,0.0004,0.0006,0.0005,0.0002,0.0006,0.0001,0.0006,0.0003


In [13]:
sandia_mods

,Vintage,Area,Material,Cells_in_Series,Parallel_Strings,Isco,Voco,Impo,Vmpo,Aisc,...,Notes,diff_Isco,percent_diff_Isco,diff_Voc,percent_diff_Voc,diff_Imp,percent_diff_Imp,diff_Vmp,percent_diff_Vmp,total_diff
Advent_Solar_AS160___2006_,2006,1.312,mc-Si,72,1,5.564,42.832,5.028,32.41,0.000537,...,Source: Sandia National Laboratories Updated 9...,3.836,0.408085,31.868,0.426613,3.872,0.435056,24.89,0.43438,1.704135
Advent_Solar_Ventura_210___2008_,2008,1.646,mc-Si,60,1,8.34,35.31,7.49,27.61,0.00077,...,Source: Sandia National Laboratories Updated 9...,1.06,0.112766,39.39,0.527309,1.41,0.158427,29.69,0.51815,1.316652
Advent_Solar_Ventura_215___2009_,2009,1.646,mc-Si,60,1,8.49,35.92,7.74,27.92,0.00082,...,Source: Sandia National Laboratories Updated 9...,0.91,0.096809,38.78,0.519143,1.16,0.130337,29.38,0.51274,1.259029
Aleo_S03_160__2007__E__,2007 (E),1.28,c-Si,72,1,5.1,43.5,4.55,35.6,0.0003,...,Source: Sandia National Laboratories Updated 9...,4.3,0.457447,31.2,0.417671,4.35,0.488764,21.7,0.378709,1.74259
Aleo_S03_165__2007__E__,2007 (E),1.28,c-Si,72,1,5.2,43.6,4.65,35.8,0.0003,...,Source: Sandia National Laboratories Updated 9...,4.2,0.446809,31.1,0.416332,4.25,0.477528,21.5,0.375218,1.715887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LG_LG290N1C_G3__2013_,2013,1.64,c-Si,60,1,9.8525,39.6117,9.2473,31.2921,0.0002,...,Source: CFV Solar Test Lab. Tested 2013. Mo...,-0.4525,-0.048138,35.0883,0.469723,-0.3473,-0.039022,26.0079,0.45389,1.010774
Sharp_NDQ235F4__2013_,2013,1.56,mc-Si,60,1,8.6739,36.8276,8.1243,29.1988,0.0006,...,Source: CFV Solar Test Lab. Tested 2013. Mo...,0.7261,0.077245,37.8724,0.506993,0.7757,0.087157,28.1012,0.490422,1.161818
Solar_Frontier_SF_160S__2013_,2013,1.22,CIS,172,1,2.0259,112.5048,1.8356,86.6752,0.0001,...,Source: CFV Solar Test Lab. Tested 2013. Mo...,7.3741,0.784479,-37.8048,-0.506088,7.0644,0.793753,-29.3752,-0.512656,2.596976
SolarWorld_Sunmodule_250_Poly__2013_,2013,1.68,mc-Si,60,1,8.3768,36.3806,7.6921,28.348,0.0006,...,Source: CFV Solar Test Lab. Tested 2013. Mo...,1.0232,0.108851,38.3194,0.512977,1.2079,0.135719,28.952,0.505271,1.262818
